In [3]:
import numpy as np
import pandas as pd

Tags = pd.read_csv("Tags.csv")
Posts = pd.read_csv("Posts.csv")
Users = pd.read_csv("Users.csv")
Votes = pd.read_csv("Votes.csv")
Comments = pd.read_csv("Comments.csv")
Badges = pd.read_csv("Badges.csv")

import sqlite3
con = sqlite3.connect("./travel.db")
Tags.to_sql("Tags", con)
Posts.to_sql("Posts", con)
Users.to_sql("Users", con)
Votes.to_sql("Votes", con)
Comments.to_sql("Comments", con)
Badges.to_sql("Badges", con)

## Zadanie1  
>Zwraca 10 użytkowników, których pytania zostały w sumie dodane do ulubionch największą ilość razy.  
>Zwraca nazwę, id, lokację użytkownika, sumę dodań do ulubionych dla wszystkich zadanych przez niego pytań,
>pytanie tego użytkownika dodane do ulubionych największą liczbę razy i liczbę dodań do ulubionych dla tego pytania.  

In [67]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie2
>Zwraca 10 pytań (id, tytuł, liczba odpowiedzi o dodatnim wyniku na dane pytanie)  
>z największą liczbą odpowiedzi o dodatnim wyniku na dane pytanie. 

In [99]:
sql = pd.read_sql_query(""" SELECT Posts.ID,
                            Posts.Title,
                            Posts2.PositiveAnswerCount
                            FROM Posts
                            JOIN (
                            SELECT
                            Posts.ParentID,
                            COUNT(*) AS PositiveAnswerCount
                            FROM Posts
                            WHERE Posts.PostTypeID=2 AND Posts.Score>0
                            GROUP BY Posts.ParentID
                            ) AS Posts2
                            ON Posts.ID=Posts2.ParentID
                            ORDER BY Posts2.PositiveAnswerCount DESC
                            LIMIT 10""", con)

pandas = Posts.query("PostTypeId == 2 and Score > 0").groupby('ParentId').size().nlargest(10).to_frame().\
join(Posts.set_index('Id'), how = 'inner').iloc[:,[15,0]].reset_index()                                                                                                
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie3
>Dla każdego roku zwraca tytuł pytania, które dostało najwięcej upvotów w danym roku, ten rok i tę liczbę upVotów. 

In [187]:
sql = pd.read_sql_query(""" SELECT Posts.Title,
                            UpVotesPerYear.Year,
                            MAX(UpVotesPerYear.Count) AS Count
                            FROM (
                            SELECT
                            PostId,
                            COUNT(*) AS Count,
                            STRFTIME('%Y', Votes.CreationDate) AS Year
                            FROM Votes
                            WHERE VoteTypeId=2
                            GROUP BY PostId, Year
                            ) AS UpVotesPerYear
                            JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
                            WHERE Posts.PostTypeId=1
                            GROUP BY Year""", con)

q_uv_y = Votes.query("VoteTypeId == 2")[['PostId', 'CreationDate']]
q_uv_y.CreationDate = q_uv_y.CreationDate.str[:4]
q_uv_y = q_uv_y.groupby(['PostId', 'CreationDate']).size().to_frame().join(\
Posts.query("PostTypeId == 1")[['Id', 'Title']].set_index('Id'), on = 'PostId', how = 'inner')
pandas = q_uv_y.groupby('CreationDate').max()[0].to_frame().reset_index().set_index(['CreationDate', 0]).join(\
q_uv_y.reset_index().set_index(['CreationDate', 0])).reset_index().iloc[:,[3,0,1]]
pandas.columns = sql.columns

sql.equals(pandas)

True

## Zadanie4
>

In [ ]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

## Zadanie5


In [ ]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

## Zadanie6


In [ ]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

## Zadanie7


In [ ]:
sql = pd.read_sql_query("""SELECT Users.DisplayName,
                           Users.Id,
                           Users.Location,
                           SUM(Posts.FavoriteCount) AS FavoriteTotal,
                           Posts.Title AS MostFavoriteQuestion,
                           MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
                           FROM Posts
                           JOIN Users ON Users.Id=Posts.OwnerUserId
                           WHERE Posts.PostTypeId=1
                           GROUP BY OwnerUserId
                           ORDER BY FavoriteTotal DESC
                           LIMIT 10""", con)

pandas = Users[['DisplayName', 'Id', 'Location']].set_index('Id').join(\
Posts.query("PostTypeId == 1")[['Title', 'FavoriteCount', 'OwnerUserId']].\
set_index('OwnerUserId'), how = 'inner').sort_values('FavoriteCount', ascending = False).groupby(level = 0).\
agg({'DisplayName': 'first', 'Location': 'first', 'FavoriteCount': ['sum', 'max'], 'Title': 'first'})
pandas.columns = pandas.columns.droplevel(0)
pandas = pandas.sort_values('sum', ascending = False).head(10).reset_index().iloc[:,[1,0,2,3,5,4]]
pandas.columns = sql.columns

sql.equals(pandas)

![pandas](https://user-images.githubusercontent.com/43205483/58571663-830ea580-823a-11e9-9f0f-03925c92cefe.png)